<a href="https://colab.research.google.com/github/graviada/colabRepo/blob/master/%D0%9C%D0%B0%D1%82%D0%A1%D1%82%D0%B0%D1%825.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install statsmodels

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.stats.stattools import durbin_watson

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [ ]:
timeSeries1 = pd.read_csv('1.txt', header = None, names=['y'])
timeSeries2 = pd.read_csv('2.txt', header = None, names=['y'])

In [ ]:
# type(timeSeries1)
# timeSeries1.head(5)
n = len(timeSeries1)
print(n)

64


In [ ]:
date_index = pd.date_range('1/9/2021', periods=n, freq='D')
# print(date_index)
x_num = pd.DataFrame([i for i in range(1, n + 1)])

In [ ]:
def draw(xdata, ydata, title):
  _layout = go.Layout(
    # plot_bgcolor='rgba(0,0,0,0)',
    title={
        'text': title,
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    yaxis_title='y',
    xaxis_title='x',
    width=1200,
    height=650)
  
  fig = go.Figure([go.Scatter(x=xdata, y=ydata, line=dict(color='darkblue', width=1.5))], layout=_layout)
  fig.show()

In [ ]:
draw(date_index, timeSeries1.y, 'First time series')

In [ ]:
draw(date_index, timeSeries2.y, 'Second time series')

SMA

In [ ]:
def draw_new(main, sup, date, title):
  trace1 = go.Scatter(
    x = date,
    y = main,
    mode = "lines",
    name = "Initial data",
    marker = dict(color = 'darkblue'))
  
  trace2 = go.Scatter(
    x = date,
    y = sup,
    mode = "lines",
    name = title,
    marker = dict(color = 'orange'))
  
  data = [trace1, trace2]

  layout = go.Layout(title = 'Line Plot',
    xaxis= dict(title= 'x', ticklen= 5, zeroline= False),
    yaxis= dict(title= 'y', ticklen= 5,zeroline= False))
  
  fig = go.Figure(data = data, layout = layout)
  fig.show()

In [ ]:
def SMA(ydata, windows, date):
  y = ydata.y
  average = []
  stats = {}

  for window in windows:
    average = []
    for i in range(window, len(y)+window):
      average.append(y[i-window:i+1].mean())
    residuals = y - average
    stats[window] = acorr_ljungbox(residuals, lags=[5])[0]

  min_value = min(stats.values())
  min_keys = [k for k in stats if stats[k] == min_value]

  average = []
  for i in range(min_keys[0], len(y)+min_keys[0]):
      average.append(y[i-min_keys[0]:i+1].mean())
  durbin = durbin_watson(y - average)

  draw_new(y, average, date, "SMA-smoothing")

  return {min_keys[0] : min_value}, durbin

In [ ]:
windows = [3, 5, 7, 9]
datas = [timeSeries1, timeSeries2]
iterator = 0
for data in datas:
  print(f'Оптимальное значение размера окна для файла {iterator + 1}: {list(SMA(data, windows, date_index)[0].keys())[0]}\nQ-статистика Льюинга-Бокса: {list(SMA(data, windows, date_index)[0].values())[0]}\nСтатистика Дарбина-Уотсона : {SMA(data, windows, date_index)[1]}')
  plt.show()
  iterator += 1

Оптимальное значение размера окна для файла 1: 3
Q-статистика Льюинга-Бокса: [53.18695928]
Статистика Дарбина-Уотсона : 0.8122116462288216


Оптимальное значение размера окна для файла 2: 9
Q-статистика Льюинга-Бокса: [6.07082744]
Статистика Дарбина-Уотсона : 1.9838330812428389


WMA

In [ ]:
weights = [0.1, 0.15, 0.2, 0.25, 0.3]
sum_weights = sum(weights)
weighted_average = timeSeries1.y.rolling(window=5, center=True).apply(lambda x: np.sum(weights*x)/sum_weights, raw=False)

In [ ]:
def WMA(ydata, windows, date):
  y = ydata.y
  average = []
  stats = {}

  for window in windows:
    weights = [np.random.rand() for _ in range(window)]
    sum_weights = sum(weights)
    average = y.rolling(window=window, center=True).apply(lambda x: np.sum(weights*x)/sum_weights, raw=False).fillna(0.1)
    residuals = y - average
    stats[window] = acorr_ljungbox(residuals, lags=[5])[0]

  min_value = min(stats.values())
  min_keys = [k for k in stats if stats[k] == min_value]

  average = []
  for i in range(min_keys[0], len(y)+min_keys[0]):
      average.append(y[i-min_keys[0]:i+1].mean())
  durbin = durbin_watson(y - average)

  draw_new(y, average, date, "WMA-smoothing")

  return {min_keys[0] : min_value}, durbin

In [ ]:
windows = [3, 5, 7, 9]
datas = [timeSeries1, timeSeries2]
iterator = 0
for data in datas:
  print(f'Оптимальное значение размера окна для файла {iterator + 1}: {list(WMA(data, windows, date_index)[0].keys())[0]}\nQ-статистика Льюинга-Бокса: {list(WMA(data, windows, date_index)[0].values())[0]}\nСтатистика Дарбина-Уотсона : {WMA(data, windows, date_index)[1]}')
  plt.show()
  iterator += 1

Оптимальное значение размера окна для файла 1: 5
Q-статистика Льюинга-Бокса: [0.5310316]
Статистика Дарбина-Уотсона : 0.5526644307272699


Оптимальное значение размера окна для файла 2: 3
Q-статистика Льюинга-Бокса: [8.04570979]
Статистика Дарбина-Уотсона : 2.25277819535021


EMA

In [ ]:
def EMA(ydata, alphas, date):
  y = ydata.y
  stats = {}

  for alpha in alphas:
    average = []
    average.append(y[0])
    for i in range(1, len(y)):
      average.append(alpha * y[i] + (1 - alpha) * average[i-1])
    residuals = y - average
    stats[alpha] = acorr_ljungbox(residuals, lags=[5])[0]

  min_value = min(stats.values())
  min_keys = [k for k in stats if stats[k] == min_value]

  average_opt = []
  average_opt.append(y[0])
  for i in range(1, len(y)):
    average_opt.append(alpha * y[i] + (1 - alpha) * average_opt[i-1])
  durbin = durbin_watson(y - average_opt)

  draw_new(y, average, date, "EMA-smoothing")

  return {min_keys[0] : min_value}, durbin

In [ ]:
alphas = list(np.arange(0.1, 0.9, 0.1))
datas = [timeSeries1, timeSeries2]
iterator = 0
for data in datas:
  print(f'Оптимальное значение параметра сглаживания для файла {iterator + 1}: {list(EMA(data, alphas, date_index)[0].keys())[0]}\nQ-статистика Льюинга-Бокса: {list(EMA(data, alphas, date_index)[0].values())}\nКритерий дарбина-Уотсона: {EMA(data, alphas, date_index)[1]}')
  plt.show()
  iterator += 1

Оптимальное значение параметра сглаживания для файла 1: 0.8
Q-статистика Льюинга-Бокса: [array([14.3542595])]
Критерий дарбина-Уотсона: 1.6354385999833967


Оптимальное значение параметра сглаживания для файла 2: 0.1
Q-статистика Льюинга-Бокса: [array([7.27228279])]
Критерий дарбина-Уотсона: 2.8415941112837912


DEMA

In [ ]:
def DEMA(series, alpha, beta):
    result = [series[0]]
    for n in range(1, len(series)):
        if n == 1:
            level, trend = series[0], series[1] - series[0]
        if n >= len(series): # прогнозируем
            value = result[-1]
        else:
            value = series[n]
        last_level, level = level, alpha*value + (1-alpha)*(level+trend)
        trend = beta*(level-last_level) + (1-beta)*trend
        result.append(level+trend)
    return result

In [ ]:
alphas = list(np.arange(0.1, 0.9, 0.1))
betas = alphas

In [ ]:
stats = {}
for data in datas:
  for alpha in alphas:
    for beta in betas:
      stats[alpha, beta] = acorr_ljungbox(data.y - DEMA(data.y, alpha, beta), lags=[5])[0]

In [ ]:
for data in datas:
  y = data.y

  min_value = min(stats.values())
  min_keys = [k for k in stats if stats[k] == min_value]

  average_opt = []
  average_opt.append(y[0])
  for i in range(1, len(y)):
    average_opt.append(alpha * y[i] + (1 - alpha) * average_opt[i-1])

  draw_new(y, average_opt, date_index, "DEMA-smoothing")
  plt.show()

  print(f'Оптимальные параметры для DEMA: alpha : {min_keys[0][0]}, beta : {min_keys[0][1]}\nQ-статистика: {acorr_ljungbox(data.y - DEMA(data.y, min_keys[0][0], min_keys[0][1]), lags=[5])[0]}\nДарбин-Уотсон: {durbin_watson(data.y - DEMA(data.y, min_keys[0][0], min_keys[0][1]))}')

Оптимальные параметры для DEMA: alpha : 0.7000000000000001, beta : 0.30000000000000004
Q-статистика: [74.28656075]
Дарбин-Уотсон: 0.6330509923924553


Оптимальные параметры для DEMA: alpha : 0.7000000000000001, beta : 0.30000000000000004
Q-статистика: [1.24517358]
Дарбин-Уотсон: 1.9152301182731872
